# Stock Prediction Model

## Dependencies

### Library Installation (if needed)

In [463]:
# Install libraries the first time
#! pip install -U yfinance pandas pathlib numpy 

### Importing Required Libraries

In [464]:
from datetime import datetime
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import numpy as np
import pathlib as Path
from tqdm import tqdm
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

## Data Preparation

In [465]:
symbols = pd.read_csv('../data/simple_screener_results.csv')['Ticker'].tolist()
symbols = pd.Series(symbols).unique()
symbols

array(['MATAS.CO', 'TRIFOR.CO', 'QQ.L', 'RNMBY', 'SAABF', 'BCKIY',
       'BAESY', 'IVSO.ST', 'NSKFF', 'GMAB', 'GN.CO', 'NVDA', 'LLY',
       'DANSKE.CO', 'CARL-B.CO', 'MAERSK-B.CO', 'RBREW.CO', 'ISS.CO',
       'DSV.CO', 'SCHO.CO', 'NETC.CO', 'JYSK.CO', 'ABBN.SW', 'TER',
       'PARKEN.CO', 'NFLX', 'TRMD-A.CO', 'STG.CO', 'NOVO-B.CO', 'EQNR',
       'NKT.CO', 'NSIS-B.CO', 'KCC.OL'], dtype=object)

#### Download and Process Historical Data

In [466]:
#symbols = ['AAPL']

In [ ]:
print_errors=True

df = pd.DataFrame()
for symbol in tqdm(symbols):
    ticker_df = pd.DataFrame()
    ticker = yf.Ticker(symbol)
    earning_dates = ticker.cash_flow.columns.tolist()
    price_data = yf.download(symbol, period='10y', rounding=False, progress=False)
    cash_flow = ticker.cash_flow
    cash_flow_columns = cash_flow[earning_dates[0]].keys().tolist()
    balance_sheet = ticker.balance_sheet
    balance_sheet_columns = balance_sheet[earning_dates[0]].keys().tolist()
    income_statement = ticker.income_stmt
    income_statement_columns = income_statement[earning_dates[0]].keys().tolist()

    for earning_date in earning_dates:
        try:
            current_ticker_data = {'Ticker': symbol, 'Date': earning_date}
        
            got_price = False
            day_offset = 0
            while(got_price==False and day_offset > -6):
                try:
                    current_ticker_data['1 Year Future Change'] = float(price_data.loc[earning_date + pd.Timedelta(days=day_offset+365,), ('Close', symbol)]) / float(price_data.loc[earning_date + pd.Timedelta(days=day_offset), ('Close', symbol)])
                    got_price = True
                except Exception:
                    day_offset += -1

            for column in cash_flow_columns:
                current_ticker_data[column] = cash_flow[earning_date][column]

            for column in balance_sheet_columns:
                current_ticker_data[column] = balance_sheet[earning_date][column]

            for column in income_statement_columns:
                current_ticker_data[column] = income_statement[earning_date][column]
                
        except Exception as error:
            if print_errors:
                print(f'Error for {symbol}: {error}')
        
        if got_price:
            ticker_df = pd.concat([ticker_df, pd.DataFrame([current_ticker_data])], ignore_index=True)

    for column in ticker_df.columns.drop(['Ticker', 'Date', '1 Year Future Change']):
        if ticker_df[column].isna().any():
            continue
        imputer = SimpleImputer(strategy='mean')
        ticker_df[column] = pd.Series(
            imputer.fit_transform(ticker_df[[column]]).flatten(),
            index=ticker_df.index
        )

    df = pd.concat([df, ticker_df], ignore_index=True)

df.to_csv('../data/earnings_data.csv', index=False)

  0%|          | 0/33 [00:00<?, ?it/s]/var/folders/ch/yfh31z5s3cq4cw_md59n4qw40000gn/T/ipykernel_28294/3691468204.py:53: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ticker_df[column].fillna(0, inplace=True)
/var/folders/ch/yfh31z5s3cq4cw_md59n4qw40000gn/T/ipykernel_28294/3691468204.py:53: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on

KeyboardInterrupt: 

### Short visualisation

In [ ]:
df

,Ticker,Date,1 Year Future Change,Free Cash Flow,Repurchase Of Capital Stock,Repayment Of Debt,Issuance Of Debt,Capital Expenditure,End Cash Position,Beginning Cash Position,...,Depreciation Amortization Depletion Income Statement,Amortization Of Intangibles Income Statement,Non Current Note Receivables,Line Of Credit,Commercial Paper,Current Notes Payable,Unrealized Gain Loss,Dueto Related Parties Current,Duefrom Related Parties Current,Preferred Stock Dividends
0,MATAS.CO,2024-03-31,1.153992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MATAS.CO,2023-03-31,1.470333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MATAS.CO,2022-03-31,0.896944,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MATAS.CO,2021-03-31,1.179800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TRIFOR.CO,2023-12-31,0.714643,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,NSIS-B.CO,2020-12-31,1.557316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,KCC.OL,2023-12-31,0.871689,136156000.0,NaN,-219511000.0,142112000.0,-12843000.0,68071000.0,64685000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,KCC.OL,2022-12-31,1.573113,95645000.0,NaN,-24049000.0,0.0,-10238000.0,64685000.0,51529000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128,KCC.OL,2021-12-31,1.437101,-73255000.0,0.0,-123041000.0,169000000.0,-119105000.0,51529000.0,65685000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Splitting and normalization

In [ ]:
test_data = df.loc[df.groupby('Ticker')['Date'].idxmax()]
training_data = df[~df.index.isin(test_data.index)]

test_data.to_csv('../data/test_data.csv', index=False)
training_data.to_csv('../data/training_data.csv', index=False)

display(test_data)
display(training_data)

,Ticker,Date,1 Year Future Change,Free Cash Flow,Repurchase Of Capital Stock,Repayment Of Debt,Issuance Of Debt,Capital Expenditure,End Cash Position,Beginning Cash Position,...,Depreciation Amortization Depletion Income Statement,Amortization Of Intangibles Income Statement,Non Current Note Receivables,Line Of Credit,Commercial Paper,Current Notes Payable,Unrealized Gain Loss,Dueto Related Parties Current,Duefrom Related Parties Current,Preferred Stock Dividends
86,ABBN.SW,2023-12-31,1.364849,3.520000e+09,-1.258000e+09,-1.567000e+09,2.586000e+09,-7.700000e+08,3.909000e+09,4.174000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,BAESY,2023-12-31,1.042759,2.803000e+09,-5.610000e+08,0.000000e+00,1.620000e+08,-9.570000e+08,4.067000e+09,3.107000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,BCKIY,2024-03-31,1.424874,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55,CARL-B.CO,2023-12-31,0.845952,7.364000e+09,-3.200000e+09,-3.725000e+09,1.527200e+10,-4.243000e+09,1.338200e+10,8.163000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,DANSKE.CO,2023-12-31,1.270911,1.401190e+11,0.000000e+00,-2.369600e+10,2.242500e+10,-1.381000e+09,3.656090e+11,2.325310e+11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,DSV.CO,2023-12-31,1.314907,1.408300e+10,-1.399700e+10,-3.270000e+08,2.120000e+08,-2.375000e+09,6.452000e+09,1.016000e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,EQNR,2023-12-31,0.819928,1.412600e+10,-5.589000e+09,-4.240000e+09,NaN,-1.057500e+10,9.641000e+09,1.557900e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,310000000.0,3.525000e+09,1.527000e+09,NaN
35,GMAB,2023-12-31,0.657903,7.004000e+09,-5.640000e+08,-9.100000e+07,NaN,-3.760000e+08,1.486700e+10,9.893000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,GN.CO,2023-12-31,0.781844,1.206000e+09,NaN,-3.273000e+09,2.540000e+08,-1.432000e+09,2.162000e+09,9.900000e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,ISS.CO,2023-12-31,1.051002,2.673000e+09,0.000000e+00,0.000000e+00,0.000000e+00,-7.190000e+08,6.093000e+09,5.214000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Ticker,Date,1 Year Future Change,Free Cash Flow,Repurchase Of Capital Stock,Repayment Of Debt,Issuance Of Debt,Capital Expenditure,End Cash Position,Beginning Cash Position,...,Depreciation Amortization Depletion Income Statement,Amortization Of Intangibles Income Statement,Non Current Note Receivables,Line Of Credit,Commercial Paper,Current Notes Payable,Unrealized Gain Loss,Dueto Related Parties Current,Duefrom Related Parties Current,Preferred Stock Dividends
1,MATAS.CO,2023-03-31,1.470333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MATAS.CO,2022-03-31,0.896944,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MATAS.CO,2021-03-31,1.179800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,TRIFOR.CO,2022-12-31,0.736962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,TRIFOR.CO,2021-12-31,0.475323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,NSIS-B.CO,2021-12-31,0.663732,3.794401e+08,NaN,-1.725094e+08,2.882776e+08,-1.667277e+08,1.294829e+08,1.587947e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,NSIS-B.CO,2020-12-31,1.557316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,KCC.OL,2022-12-31,1.573113,9.564500e+07,NaN,-2.404900e+07,0.000000e+00,-1.023800e+07,6.468500e+07,5.152900e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128,KCC.OL,2021-12-31,1.437101,-7.325500e+07,0.0,-1.230410e+08,1.690000e+08,-1.191050e+08,5.152900e+07,6.568500e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
X_train = training_data.drop(columns=['Ticker', 'Date', '1 Year Future Change'])
y_train = training_data['1 Year Future Change']
X_test = test_data.drop(columns=['Ticker', 'Date', '1 Year Future Change'])
y_test = test_data['1 Year Future Change']

display(X_train)
display(y_train)

,Free Cash Flow,Repurchase Of Capital Stock,Repayment Of Debt,Issuance Of Debt,Capital Expenditure,End Cash Position,Beginning Cash Position,Effect Of Exchange Rate Changes,Changes In Cash,Financing Cash Flow,...,Depreciation Amortization Depletion Income Statement,Amortization Of Intangibles Income Statement,Non Current Note Receivables,Line Of Credit,Commercial Paper,Current Notes Payable,Unrealized Gain Loss,Dueto Related Parties Current,Duefrom Related Parties Current,Preferred Stock Dividends
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,3.794401e+08,NaN,-1.725094e+08,2.882776e+08,-1.667277e+08,1.294829e+08,1.587947e+08,7.126266e+06,-3.643808e+07,-2.132501e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,9.564500e+07,NaN,-2.404900e+07,0.000000e+00,-1.023800e+07,6.468500e+07,5.152900e+07,NaN,1.315600e+07,-8.248900e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128,-7.325500e+07,0.0,-1.230410e+08,1.690000e+08,-1.191050e+08,5.152900e+07,6.568500e+07,-7.420000e+05,-1.341300e+07,4.625400e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


1      1.470333
2      0.896944
3      1.179800
5      0.736962
6      0.475323
         ...   
124    0.663732
125    1.557316
127    1.573113
128    1.437101
129    1.409652
Name: 1 Year Future Change, Length: 97, dtype: float64

## Model Training

In [ ]:
model = MLPClassifier(
    solver='adam',
    alpha=1e-4,
    hidden_layer_sizes=(50, 50, 50),
    shuffle=False,
    random_state=42,
    tol=0.0001,
    verbose=True,
    max_iter=50,
    batch_size=1024,
    learning_rate='adaptive',
    n_iter_no_change=4
)
model.fit(X_train, y_train)

ValueError: Input X contains NaN.
MLPClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

## Testing and benchmarking

## Predictions